## Asymmetric Semantic Search

For asymmetric semantic search, you usually have a short query (like a question or some keywords) and you want to find a longer paragraph answering the query. `sentence-transformers/msmarco-distilbert-base-v4` seems to be good for this task. Meh.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example.pdf"
loader = PyPDFLoader(file_path )
docs = loader.load()

print(len(docs))

17


In [12]:
from sentence_transformers import SentenceTransformer
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-v4")


text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=embeddings
)

In [5]:
retriever = vectorstore.as_retriever()

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

model = ChatOllama(
  model="mistral-nemo"
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Col profilo semplice, a quanto ammontano le commissioni di trading su EXM?"})

results

{'input': 'Col profilo semplice, a quanto ammontano le commissioni di trading su EXM?',
 'context': [Document(id='f6372a6e-6329-4a4a-87c2-12f952db2384', metadata={'source': './example.pdf', 'page': 15}, page_content='ALL1\n0524\n!\n!  \nINFORMAZIONI SUGLI ONERI E SUI COSTI  \n!\nDi seguito sono riportate le commissioni e le condizioni \neconomiche in vigore al  2 maggio 2024. Per successive \nmodifiche si rinvia al sito www.directa.it. \n \nCOMMISSIONI DI TRADING SUI DIVERSI MERCATI  \n \nEXM (ex MTA), EGM, MIV, ETFplus, GEM  \nProfili alternativi: \n•!Semplice: 5! per ordine eseguito \n•!Dinamica*: da 8 a 1,5!  \n•!Variabile: 1,9 per mille per ordine eseguito, con un  \nmassimo di 18! e un minimo di 1,5! (il minimo è di 5! per il \nmercato GEM) per ordini fino a 500.000!   \n \nATFund'),
  Document(id='e01b7ea7-7fcc-4170-8640-7221b4f64201', metadata={'source': './example.pdf', 'page': 2}, page_content="quale un servizio è prestato. \n \nINCENTIVI \nIn conformità a quanto previsto dall

In [ ]:
print(results["context"][0].metadata) # it's page + 1 actually

{'source': './example.pdf', 'page': 15}


In [10]:
rag_chain.invoke({"input": "Quanto costa una operazioni fuori mercato con controparte esterna a directa?"})

{'input': 'Quanto costa una operazioni fuori mercato con controparte esterna a directa?',
 'context': [Document(id='e01b7ea7-7fcc-4170-8640-7221b4f64201', metadata={'source': './example.pdf', 'page': 2}, page_content="quale un servizio è prestato. \n \nINCENTIVI \nIn conformità a quanto previsto dalla normativa MIFID \nII, si segnala che per quei Cli enti che apriranno un co n-\nto con un intermediario convenzi onato, una parte delle \ncommissioni che gli stessi pag heranno verrà da Directa \nretrocessa all'intermediario stesso e/o alla società Ce n-\ntrale Trading S.r.l., per un importo che potrà variare in \nfunzione di diversi parametri, senza eccedere, in qu a-\nlunque caso, il  limite massimo del 28% della c ommis-\nsione. \nNel caso in cui il cliente effettui operazioni di compr a-"),
  Document(id='543a85aa-f5eb-4a92-ab30-792a5c6126e1', metadata={'source': './example.pdf', 'page': 7}, page_content='denza spot che si rinnova giorno per giorno . Pertanto, \nad ogni chiu sura di gi